In [ ]:
pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator
import warnings
import matplotlib.dates as mdates
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("deep")


In [ ]:
def currency_formatter(x, pos):
    return f'₹{x:,.0f}'

In [ ]:

currency_format = FuncFormatter(currency_formatter)

In [ ]:
# Data Download
end_date = datetime.now()
start_date = datetime(end_date.year - 5, end_date.month, end_date.day)
print(f"Fetching data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

In [ ]:
reliance_data = yf.download('RELIANCE.NS', start=start_date, end=end_date)
tcs_data = yf.download('TCS.NS', start=start_date, end=end_date)
adani_data = yf.download('ADANIENT.NS', start=start_date, end=end_date)

In [ ]:
print(f"RELIANCE data shape: {reliance_data.shape}")
print(f"TCS data shape: {tcs_data.shape}")
print(f"ADANIENT data shape: {adani_data.shape}")

In [ ]:
reliance_data.head()

In [ ]:
reliance_data.tail()

In [ ]:
# Data Preprocessing
def calculate_overnight_returns(df):
    """
    Calculate overnight returns from previous close to current open.
    Extremely simple approach to avoid any indexing issues.
    """
    # Create a new empty DataFrame with the same index
    result = pd.DataFrame(index=df.index)
    
    # Copying the original data
    result['Open'] = df['Open']
    result['High'] = df['High']
    result['Low'] = df['Low']
    result['Close'] = df['Close']
    result['Volume'] = df['Volume']
    
    # Calculate previous close
    result['Prev_Close'] = result['Close'].shift(1)
    
    # Calculate returns using simple methods
    # Handle potential divisions by zero by replacing zeros with NaN
    prev_close_safe = result['Prev_Close'].replace(0, np.nan)
    open_safe = result['Open'].replace(0, np.nan)
    
    # Calculate the returns
    result['Overnight_Return'] = result['Open'] / prev_close_safe - 1
    result['Day_Return'] = result['Close'] / open_safe - 1
    result['Close_to_Close_Return'] = result['Close'] / result['Close'].shift(1).replace(0, np.nan) - 1
    
    return result

In [ ]:
# Process both datasets
reliance_data = calculate_overnight_returns(reliance_data)
tcs_data = calculate_overnight_returns(tcs_data)
adani_data = calculate_overnight_returns(adani_data)

In [ ]:

# Remove the first row which has NaN values due to the shift
reliance_data = reliance_data.iloc[1:]
tcs_data = tcs_data.iloc[1:]
adani_data = adani_data.iloc[1:]


## Data Preprocessing


In [ ]:
reliance_data[['Open', 'Close', 'Prev_Close', 'Overnight_Return', 'Day_Return', 'Close_to_Close_Return']].head()

reliance_data[['Open', 'Close', 'Prev_Close', 'Overnight_Return', 'Day_Return', 'Close_to_Close_Return']].tail()

In [ ]:
# Strategy Implementation
initial_capital = 100000  
allocation_per_stock = initial_capital / 3  

In [ ]:
def overnight_strategy(df, allocation):
    """
    Implement a strategy that buys at close and sells at next day's open
    
    Parameters:
    df (DataFrame): Stock data with Overnight_Return column
    allocation (float): Initial capital allocation
    
    Returns:
    DataFrame: DataFrame with strategy performance metrics
    """
    # Create a copy of the dataframe to avoid modifying the original
    strategy_df = df.copy()
    
    # Calculate the number of shares we can buy with our allocation
    # For simplicity, we'll assume fractional shares are possible
   
    if (['prev_close'] -['prev_open']/['prev_close']) > 0.01:
        strategy_df['Shares'] = allocation / strategy_df['Prev_Close']
        
    # Calculate daily PnL based on overnight returns
    strategy_df['Daily_PnL'] = strategy_df['Shares'] * strategy_df['Prev_Close'] * strategy_df['Overnight_Return']
    
    # Calculate cumulative PnL
    strategy_df['Cumulative_PnL'] = strategy_df['Daily_PnL'].cumsum()
    
    # Calculate strategy equity curve (initial allocation + cumulative PnL)
    strategy_df['Equity'] = allocation + strategy_df['Cumulative_PnL']
    
    # Calculate daily returns of the strategy (based on equity)
    strategy_df['Strategy_Return'] = strategy_df['Daily_PnL'] / strategy_df['Equity'].shift(1)
    strategy_df['Strategy_Return'].fillna(0, inplace=True)  # Fill NaN for first day
    
    return strategy_df

In [ ]:
def overnight_strategy(df, allocation):
    """
    Buy at yesterday's close and sell at today's open, but only
    on days when overnight return > 1%.
    
    Parameters:
      df (DataFrame): must contain 'Prev_Open' and 'Prev_Close'.
      allocation (float): capital to deploy on each qualifying day.
    
    Returns:
      DataFrame: with columns Shares, Daily_PnL, Cumulative_PnL, Equity, Strategy_Return
    """
    strategy_df = df.copy()
    
    # 1) compute overnight return if not already there
    if 'Overnight_Return' not in strategy_df:
        strategy_df['Overnight_Return'] = (
            (strategy_df['Prev_Close'] - strategy_df['Prev_Open'])
            / strategy_df['Prev_Open']
        )
    
    # 2) build the mask of days where return > 1%
    mask = strategy_df['Overnight_Return'] > 0.01
    
    # 3) allocate shares only on those days
    strategy_df['Shares'] = 0.0
    strategy_df.loc[mask, 'Shares'] = allocation / strategy_df.loc[mask, 'Prev_Close']
    
    # 4) PnL and equity curve
    strategy_df['Daily_PnL']      = strategy_df['Shares'] * strategy_df['Prev_Close'] * strategy_df['Overnight_Return']
    strategy_df['Cumulative_PnL'] = strategy_df['Daily_PnL'].cumsum()
    strategy_df['Equity']         = allocation + strategy_df['Cumulative_PnL']
    
    # 5) daily strategy returns
    strategy_df['Strategy_Return'] = strategy_df['Equity'].pct_change().fillna(0)
    
    return strategy_df


In [ ]:
# Run the strategy for both stocks
reliance_strategy = overnight_strategy(reliance_data, allocation_per_stock)
tcs_strategy = overnight_strategy(tcs_data, allocation_per_stock)
adani_strategy = overnight_strategy(adani_data, allocation_per_stock)


In [ ]:
# Combine the equity curves to get the total portfolio performance
combined_equity = pd.DataFrame({
    'Date': reliance_strategy.index,
    'RELIANCE_Equity': reliance_strategy['Equity'],
    'TCS_Equity': tcs_strategy['Equity'],
    'ADANI_Equity': adani_strategy['Equity'],
})

## Strategy Implementation

In [ ]:
combined_equity.set_index('Date', inplace=True)
combined_equity['Total_Equity'] = combined_equity['RELIANCE_Equity'] + combined_equity['TCS_Equity'] + combined_equity['ADANI_Equity']
combined_equity['Daily_Return'] = combined_equity['Total_Equity'].pct_change()




In [ ]:
combined_equity.head()

In [ ]:
# Calculate key performance metrics
def calculate_performance_metrics(equity_df, risk_free_rate=0.06/252):  # 6% annual risk-free rate for India
    """
    Calculate performance metrics for the strategy
    
    Parameters:
    equity_df (DataFrame): DataFrame with equity curve
    risk_free_rate (float): Daily risk-free rate, default is 6% annual converted to daily for India
    
    Returns:
    dict: Dictionary with performance metrics
    """
    daily_returns = equity_df['Daily_Return'].dropna()
    
    # Calculate annualized return
    total_days = len(daily_returns)
    total_years = total_days / 252  # Assuming 252 trading days in a year
    
    start_equity = equity_df['Total_Equity'].iloc[0]
    end_equity = equity_df['Total_Equity'].iloc[-1]
    
    total_return = (end_equity / start_equity) - 1
    annualized_return = ((1 + total_return) ** (1 / total_years)) - 1
    
    # Calculate Sharpe Ratio
    excess_returns = daily_returns - risk_free_rate
    sharpe_ratio = (excess_returns.mean() / daily_returns.std()) * np.sqrt(252)
    
    # Calculate maximum drawdown
    equity_series = equity_df['Total_Equity']
    rolling_max = equity_series.cummax()
    drawdown = (equity_series / rolling_max) - 1
    max_drawdown = drawdown.min()
    
    # Calculate winning days and losing days
    winning_days = (daily_returns > 0).sum()
    losing_days = (daily_returns < 0).sum()
    win_rate = winning_days / (winning_days + losing_days)
    
    # Calculate profit factor
    gross_profit = daily_returns[daily_returns > 0].sum()
    gross_loss = abs(daily_returns[daily_returns < 0].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else float('inf')
    
    # Average win and average loss
    avg_win = daily_returns[daily_returns > 0].mean() if winning_days > 0 else 0
    avg_loss = daily_returns[daily_returns < 0].mean() if losing_days > 0 else 0
    
    return {
        'Total Return': total_return,
        'Annualized Return': annualized_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Winning Days': winning_days,
        'Losing Days': losing_days,
        'Win Rate': win_rate,
        'Profit Factor': profit_factor,
        'Average Win': avg_win,
        'Average Loss': avg_loss
    }

# Calculate performance metrics
performance = calculate_performance_metrics(combined_equity)

In [ ]:
# Print performance metrics
print("\nPerformance Metrics:")
for metric, value in performance.items():
    if metric in ['Total Return', 'Annualized Return', 'Win Rate']:
        print(f"{metric}: {value*100:.2f}%")
    elif isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

In [ ]:
# Define a function to analyze trades
def analyze_trades(strategy_df, stock_name):
    """
    Analyze the trading performance for a specific stock
    
    Parameters:
    strategy_df (DataFrame): Strategy dataframe with Daily_PnL
    stock_name (str): Name of the stock for display
    
    Returns:
    dict: Dictionary of trade analytics
    """
    daily_pnl = strategy_df['Daily_PnL']
    
    # Count winning, losing, and breakeven trades
    winning_trades = (daily_pnl > 0).sum()
    losing_trades = (daily_pnl < 0).sum()
    breakeven_trades = (daily_pnl == 0).sum()
    
    # Calculate win rate
    total_trades = len(daily_pnl)
    win_rate = winning_trades / total_trades if total_trades > 0 else 0
    
    # Calculate gross profit, gross loss, and net profit
    gross_profit = daily_pnl[daily_pnl > 0].sum()
    gross_loss = daily_pnl[daily_pnl < 0].sum()
    net_profit = gross_profit + gross_loss
    
    # Calculate profit factor
    profit_factor = abs(gross_profit / gross_loss) if gross_loss != 0 else float('inf')
    
    # Calculate average win and average loss
    avg_win = daily_pnl[daily_pnl > 0].mean() if winning_trades > 0 else 0
    avg_loss = daily_pnl[daily_pnl < 0].mean() if losing_trades > 0 else 0
    
    # Calculate average trade
    avg_trade = daily_pnl.mean()
    
    # Calculate Sharpe ratio
    sharpe_ratio = daily_pnl.mean() / daily_pnl.std() * np.sqrt(252) if daily_pnl.std() != 0 else 0
    
    # Calculate winning and losing streaks
    daily_pnl_binary = daily_pnl > 0
    
    # Initialize variables for the streaks calculation
    current_streak = 1
    max_win_streak = 0
    max_loss_streak = 0
    in_win_streak = daily_pnl_binary.iloc[0]
    
    # Calculate streaks
    for i in range(1, len(daily_pnl_binary)):
        if daily_pnl_binary.iloc[i] == daily_pnl_binary.iloc[i-1]:
            current_streak += 1
        else:
            if in_win_streak:
                max_win_streak = max(max_win_streak, current_streak)
            else:
                max_loss_streak = max(max_loss_streak, current_streak)
            current_streak = 1
            in_win_streak = daily_pnl_binary.iloc[i]
    
    # Check the last streak
    if in_win_streak:
        max_win_streak = max(max_win_streak, current_streak)
    else:
        max_loss_streak = max(max_loss_streak, current_streak)
    
    return {
        'Stock': stock_name,
        'Total Trading Days': total_trades,
        'Winning Trades': winning_trades,
        'Losing Trades': losing_trades,
        'Breakeven Trades': breakeven_trades,
        'Win Rate': win_rate,
        'Gross Profit': gross_profit,
        'Gross Loss': gross_loss,
        'Net Profit': net_profit,
        'Profit Factor': profit_factor,
        'Average Win': avg_win,
        'Average Loss': avg_loss,
        'Average Trade': avg_trade,
        'Sharpe Ratio': sharpe_ratio,
        'Max Win Streak': max_win_streak,
        'Max Loss Streak': max_loss_streak
    }

In [ ]:
# Calculate key performance metrics
def calculate_performance_metrics(equity_df, risk_free_rate=0.02/252):
    """
    Calculate performance metrics for the strategy
    
    Parameters:
    equity_df (DataFrame): DataFrame with equity curve
    risk_free_rate (float): Daily risk-free rate, default is 2% annual converted to daily
    
    Returns:
    dict: Dictionary with performance metrics
    """
    daily_returns = equity_df['Daily_Return'].dropna()
    
    # Calculate annualized return
    total_days = len(daily_returns)
    total_years = total_days / 252  # Assuming 252 trading days in a year
    
    start_equity = equity_df['Total_Equity'].iloc[0]
    end_equity = equity_df['Total_Equity'].iloc[-1]
    
    total_return = (end_equity / start_equity) - 1
    annualized_return = ((1 + total_return) ** (1 / total_years)) - 1
    
    # Calculate Sharpe Ratio
    excess_returns = daily_returns - risk_free_rate
    sharpe_ratio = (excess_returns.mean() / daily_returns.std()) * np.sqrt(252)
    
    # Calculate maximum drawdown
    equity_series = equity_df['Total_Equity']
    rolling_max = equity_series.cummax()
    drawdown = (equity_series / rolling_max) - 1
    max_drawdown = drawdown.min()
    
    # Calculate winning days and losing days
    winning_days = (daily_returns > 0).sum()
    losing_days = (daily_returns < 0).sum()
    win_rate = winning_days / (winning_days + losing_days)
    
    # Calculate profit factor
    gross_profit = daily_returns[daily_returns > 0].sum()
    gross_loss = abs(daily_returns[daily_returns < 0].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else float('inf')
    
    # Average win and average loss
    avg_win = daily_returns[daily_returns > 0].mean() if winning_days > 0 else 0
    avg_loss = daily_returns[daily_returns < 0].mean() if losing_days > 0 else 0
    
    return {
        'Total Return': total_return,
        'Annualized Return': annualized_return,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Winning Days': winning_days,
        'Losing Days': losing_days,
        'Win Rate': win_rate,
        'Profit Factor': profit_factor,
        'Average Win': avg_win,
        'Average Loss': avg_loss
    }

In [ ]:
# Create a function for plotting the equity curve
def plot_equity_curve(equity_df):
    """
    Plot the equity curve for the strategy
    
    Parameters:
    equity_df (DataFrame): DataFrame with equity columns
    """
    plt.figure(figsize=(14, 7))
    
    plt.plot(equity_df.index, equity_df['RELIANCE_Equity'], label='RELIANCE')
    plt.plot(equity_df.index, equity_df['TCS_Equity'], label='TCS')
    plt.plot(equity_df.index, equity_df['ADANI_Equity'], label='ADANI')
    plt.plot(equity_df.index, equity_df['Total_Equity'], label='Combined Portfolio', linewidth=2)
    
    plt.gca().yaxis.set_major_formatter(currency_format)
    plt.title('Overnight Returns Strategy - Equity Curve', fontsize=15)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Portfolio Value', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    return plt

# Plot the equity curve
plot_equity_curve(combined_equity)
plt.show()

In [ ]:
def plot_drawdown(equity_df):
    """
    Plot drawdown chart for the strategy
    """
    equity_series = equity_df['Total_Equity']
    rolling_max = equity_series.cummax()
    drawdown = (equity_series / rolling_max) - 1
    
    plt.figure(figsize=(14, 5))
    
    plt.plot(equity_df.index, drawdown, color='red', linewidth=1.5)
    plt.fill_between(equity_df.index, drawdown, 0, color='red', alpha=0.3)
    
    plt.title('Strategy Drawdown', fontsize=15)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Drawdown', fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    
    return plt
# Plot drawdown
plot_drawdown(combined_equity)
plt.show()


In [ ]:
# Analyze individual stock performance
reliance_trade_analytics = analyze_trades(reliance_strategy, 'RELIANCE')
tcs_trade_analytics = analyze_trades(tcs_strategy, 'TCS')
adani_trade_analytics = analyze_trades(adani_strategy, 'ADANI')


In [ ]:
print("\nRELIANCE Trade Analytics:")
for metric, value in reliance_trade_analytics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

print("\nTCS Trade Analytics:")
for metric, value in tcs_trade_analytics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")
        
print("\nADANI Trade Analytics:")
for metric, value in adani_trade_analytics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")


In [ ]:
def analyze_overnight_vs_day_returns(stock_data, stock_name):
    """
    Analyze the long-term difference between overnight and day returns
    
    Parameters:
    stock_data (DataFrame): DataFrame with Overnight_Return and Day_Return columns
    stock_name (str): Name of the stock for display purposes
    
    Returns:
    dict: Dictionary with analysis results
    """
    # Remove NaN values for clean analysis
    clean_data = stock_data.dropna(subset=['Overnight_Return', 'Day_Return'])
    
    # Calculate basic statistics
    overnight_mean = clean_data['Overnight_Return'].mean()
    day_mean = clean_data['Day_Return'].mean()
    overnight_std = clean_data['Overnight_Return'].std()
    day_std = clean_data['Day_Return'].std()
    
    # Calculate annualized returns (assuming 252 trading days per year)
    overnight_annual = ((1 + overnight_mean) ** 252) - 1
    day_annual = ((1 + day_mean) ** 252) - 1
    
    # Calculate Sharpe ratio (assuming 0 risk-free rate for simplicity)
    overnight_sharpe = overnight_mean / overnight_std * np.sqrt(252)
    day_sharpe = day_mean / day_std * np.sqrt(252)
    
    # Calculate percent of positive days
    overnight_positive = (clean_data['Overnight_Return'] > 0).mean() * 100
    day_positive = (clean_data['Day_Return'] > 0).mean() * 100
    
    # Run t-test to check if the difference is statistically significant
    from scipy import stats
    t_stat, p_value = stats.ttest_ind(
        clean_data['Overnight_Return'],
        clean_data['Day_Return'],
        equal_var=False  # Use Welch's t-test assuming unequal variances
    )
#Null hypothesis : H0 We are assuming that overnight_returns are less than day_returns
#Alternative hypothesis : H1 We are assuming that overnight returns are greater than day_returns

#New Hypothesis testing becomes:
#Null hypothesis H0: overnight_returns and day_returns are the same and 
#HA (H1): overnight_returns and day_returns are not the same   

# t test: 
#x1: mean of overnight returns
#x2: mean of day returns
#Std1 : std of overnight returns
# std2: std of day returns 

#n1 : Sample size of overnight returns

# t = x1 -x2 /sqrt(std1- std2/)
    return {
        'stock': stock_name,
        'sample_size': len(clean_data),
        'overnight_mean': overnight_mean,
        'day_mean': day_mean,
        'overnight_std': overnight_std,
        'day_std': day_std,
        'overnight_annual': overnight_annual,
        'day_annual': day_annual,
        'overnight_sharpe': overnight_sharpe,
        'day_sharpe': day_sharpe,
        'overnight_positive': overnight_positive,
        'day_positive': day_positive,
        't_stat': t_stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    }


In [ ]:
reliance_analysis = analyze_overnight_vs_day_returns(reliance_data, 'RELIANCE')
tcs_analysis = analyze_overnight_vs_day_returns(tcs_data, 'TCS')
adani_analysis = analyze_overnight_vs_day_returns(adani_data, 'ADANI')


In [ ]:
# Print the analysis results in a formatted table
print(f"\n{'=' * 80}")
print(f"LONG-TERM OVERNIGHT VS DAY RETURNS ANALYSIS - ENTIRE DATASET")
print(f"{'=' * 80}")

In [ ]:
metrics = [
    ('Sample Size (Trading Days)', 'sample_size', '{:,d}'),
    ('Average Overnight Return', 'overnight_mean', '{:.4%}'),
    ('Average Day Return', 'day_mean', '{:.4%}'),
    ('Difference (Overnight - Day)', lambda x: x['overnight_mean'] - x['day_mean'], '{:.4%}'),
    ('Standard Deviation - Overnight', 'overnight_std', '{:.4%}'),
    ('Standard Deviation - Day', 'day_std', '{:.4%}'),
    ('Annualized Overnight Return', 'overnight_annual', '{:.2%}'),
    ('Annualized Day Return', 'day_annual', '{:.2%}'),
    ('Sharpe Ratio - Overnight', 'overnight_sharpe', '{:.2f}'),
    ('Sharpe Ratio - Day', 'day_sharpe', '{:.2f}'),
    ('% Positive Overnight Returns', 'overnight_positive', '{:.2f}%'),
    ('% Positive Day Returns', 'day_positive', '{:.2f}%'),
    ('T-Statistic', 't_stat', '{:.2f}'),
    ('P-Value', 'p_value', '{:.4f}'),
    ('Statistically Significant', 'significant', '{}')
]

# Print the header
print(f"{'Metric':<35} | {'RELIANCE':<15} | {'TCS':<15} | {'ADANI':<15}")
print(f"{'-' * 35} | {'-' * 15} | {'-' * 15} | {'-' * 15}")

In [ ]:
# Print each metric
for desc, key, fmt in metrics:
    if callable(key):
        reliance_val = fmt.format(key(reliance_analysis))
        tcs_val = fmt.format(key(tcs_analysis))
        adani_val = fmt.format(key(adani_analysis))
    else:
        reliance_val = fmt.format(reliance_analysis[key])
        tcs_val = fmt.format(tcs_analysis[key])
        adani_val = fmt.format(adani_analysis[key])
    print(f"{desc:<35} | {reliance_val:<15} | {tcs_val:<15} | {adani_val:<15}")

In [ ]:
plt.figure(figsize=(30, 10))

plt.subplots_adjust(wspace=0.35, left=0.05, right=0.95)

plt.xticks(rotation=45, ha='right')  # Rotate date labels
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as year-month
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Show only yearly ticks

plt.tight_layout(rect=[0, 0, 1, 0.95]) 


ax = plt.gca()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0f}x'.format(x)))

In [ ]:

# Create a larger figure for better spacing
plt.figure(figsize=(18, 6))

# RELIANCE cumulative returns
plt.subplot(1, 3, 1)
reliance_overnight_cum = (1 + reliance_data['Overnight_Return'].fillna(0)).cumprod() - 1
reliance_day_cum = (1 + reliance_data['Day_Return'].fillna(0)).cumprod() - 1
reliance_total_cum = (1 + reliance_data['Close_to_Close_Return'].fillna(0)).cumprod() - 1

plt.plot(reliance_data.index, reliance_overnight_cum, 'b-', label='Overnight', linewidth=2)
plt.plot(reliance_data.index, reliance_day_cum, 'r-', label='Day', linewidth=2)
plt.plot(reliance_data.index, reliance_total_cum, 'g-', label='Total (Close-to-Close)', linewidth=1.5, alpha=0.8)
plt.title('RELIANCE: Cumulative Returns', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator())

# TCS cumulative returns
plt.subplot(1, 3, 2)
tcs_overnight_cum = (1 + tcs_data['Overnight_Return'].fillna(0)).cumprod() - 1
tcs_day_cum = (1 + tcs_data['Day_Return'].fillna(0)).cumprod() - 1
tcs_total_cum = (1 + tcs_data['Close_to_Close_Return'].fillna(0)).cumprod() - 1

plt.plot(tcs_data.index, tcs_overnight_cum, 'b-', label='Overnight', linewidth=2)
plt.plot(tcs_data.index, tcs_day_cum, 'r-', label='Day', linewidth=2)
plt.plot(tcs_data.index, tcs_total_cum, 'g-', label='Total (Close-to-Close)', linewidth=1.5, alpha=0.8)
plt.title('TCS: Cumulative Returns', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator())

# ADANI cumulative returns
plt.subplot(1, 3, 3)
adani_overnight_cum = (1 + adani_data['Overnight_Return'].fillna(0)).cumprod() - 1
adani_day_cum = (1 + adani_data['Day_Return'].fillna(0)).cumprod() - 1
adani_total_cum = (1 + adani_data['Close_to_Close_Return'].fillna(0)).cumprod() - 1

plt.plot(adani_data.index, adani_overnight_cum, 'b-', label='Overnight', linewidth=2)
plt.plot(adani_data.index, adani_day_cum, 'r-', label='Day', linewidth=2)
plt.plot(adani_data.index, adani_total_cum, 'g-', label='Total (Close-to-Close)', linewidth=1.5, alpha=0.8)
plt.title('ADANI: Cumulative Returns', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return', fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator())

# Create a single legend for the entire figure
handles, labels = plt.gca().get_legend_handles_labels()
fig = plt.gcf()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05),
          fancybox=True, shadow=True, ncol=3, fontsize=12)

# Add an overall title
fig.suptitle('Overnight vs Day Trading Returns Comparison (2020-2025)', 
             fontsize=16, fontweight='bold', y=0.98)

# Adjust spacing
plt.subplots_adjust(top=0.85, bottom=0.20, wspace=0.3)

plt.tight_layout(rect=[0, 0.1, 1, 0.9])
plt.show()

In [ ]:
print("\nConclusion:")
print("=" * 80)
print("This analysis has implemented an overnight returns strategy for Indian stocks")
print("(RELIANCE, TCS, and ADANI) over a 5-year period. The strategy buys at market close")
print("and sells at the next day's open, capturing the overnight price movements.")
print("\nKey findings:")
print("1. The strategy generated a total return of {:.2%} over the period".format(performance['Total Return']))
print("2. Annualized return was {:.2%}".format(performance['Annualized Return']))
print("3. Maximum drawdown reached {:.2%}".format(performance['Max Drawdown']))
print("4. The win rate was {:.2%}".format(performance['Win Rate']))
print("\nComparison between stocks:")
print("- RELIANCE had a net profit of ₹{:,.2f}".format(reliance_trade_analytics['Net Profit']))
print("- TCS had a net profit of ₹{:,.2f}".format(tcs_trade_analytics['Net Profit']))
print("- ADANI had a net profit of ₹{:,.2f}".format(adani_trade_analytics['Net Profit']))
print("\nOvernight vs Day returns analysis showed:")
if reliance_analysis['overnight_mean'] > reliance_analysis['day_mean']:
    print("- For RELIANCE, overnight returns ({:.2%}) were higher than day returns ({:.2%})".format(
        reliance_analysis['overnight_mean'], reliance_analysis['day_mean']))
else:
    print("- For RELIANCE, day returns ({:.2%}) were higher than overnight returns ({:.2%})".format(
        reliance_analysis['day_mean'], reliance_analysis['overnight_mean']))
    
if tcs_analysis['overnight_mean'] > tcs_analysis['day_mean']:
    print("- For TCS, overnight returns ({:.2%}) were higher than day returns ({:.2%})".format(
        tcs_analysis['overnight_mean'], tcs_analysis['day_mean']))
else:
    print("- For TCS, day returns ({:.2%}) were higher than overnight returns ({:.2%})".format(
        tcs_analysis['day_mean'], tcs_analysis['overnight_mean']))
        
if adani_analysis['overnight_mean'] > adani_analysis['day_mean']:
    print("- For ADANI, overnight returns ({:.2%}) were higher than day returns ({:.2%})".format(
        adani_analysis['overnight_mean'], adani_analysis['day_mean']))
else:
    print("- For ADANI, day returns ({:.2%}) were higher than overnight returns ({:.2%})".format(
        adani_analysis['day_mean'], adani_analysis['overnight_mean']))